# Generating point geometries from a table of point-events with known Route IDs and mile markers

Suppose we have a table of point events with Route ID and mile marker information about where each event happened, but with no associated geometries. 
Suppose further that we also have the linework of the reference routes. 

We can use `linref` to generate geometries for each of the rows of the point data. The example below shows the step-by-step process.

In [7]:
# Importing relevant libraries
from shapely.geometry import LineString
import pandas as pd
import geopandas as gpd
import linref as lr

First, let's create the sample DataFrames and GeoDataFrames we'll be using throughout this example.

In [8]:
# `point_df` is the DataFrame containing the points with Route ID and mile marker info but without any
# actual geometries 
point_df = pd.DataFrame({'Point_ID':[1,2,3,4],
                         'Route_ID':['A','A','B','B'],
                         'BEG':[1,2,2,3]})

# `ref_gdf` is the GeoDataFrame that contains the reference linework of the roadway network
ref_gdf = gpd.GeoDataFrame({'Route_ID':['A','B'],
                            'BEG':[0,0],
                            'END':[5,5],
                            'geometry':[LineString(((0,0),(5,0))),
                                        LineString(((0,2),(5,2)))]})

The first processing step is to create `EventCollection` objects for both sets:

In [9]:
# For the reference object, notice how we specify the `beg` and `end` parameters because we are dealing with a link-based object.
# The idea here is that the `beg` and `end` columns indicate which columns contain the start and end mile marker information for 
# each link. 
ref_ec = lr.EventsCollection(ref_gdf,
                             keys=['Route_ID'],
                             beg='BEG',
                             end='END',
                             geom='geometry')

# For the point object, notice how we only specify the `beg` (an not the `end`) parameter. this is because we are dealing with a 
# line-based object, which only has a start mile marker.  
point_ec = lr.EventsCollection(point_df, 
                               keys=['Route_ID'],
                               beg='BEG')

We then need to build the routes for the reference `EventsCollection` object.
This step directs the `linref` library to perform some internal processing steps that are required for GIS-based operations.

In [10]:
ref_ec.build_routes()

Then, we can create the new geometries by merging the `point_ec` with the `ref_ec` and applying the `interpolate()` method. 
This will generate a numpy array of `Point` geometries, which can easily be added to your DataFrame. 

In [11]:
em = point_ec.merge(ref_ec)
new_geoms = em.interpolate()
print(new_geoms)

0    POINT (1 0)
1    POINT (2 0)
2    POINT (2 2)
3    POINT (3 2)
Name: route, dtype: object


Finally, if you want to want to create a full-fledged GeoDataFrame, you can just make a copy of your original input DataFrame and add the new array of points as the geometry column.

In [14]:
point_gdf = point_df.copy()
point_gdf['geometry'] = new_geoms

point_gdf = gpd.GeoDataFrame(point_gdf, 
                             geometry='geometry')

print(point_gdf)

   Point_ID Route_ID  BEG                 geometry
0         1        A    1  POINT (1.00000 0.00000)
1         2        A    2  POINT (2.00000 0.00000)
2         3        B    2  POINT (2.00000 2.00000)
3         4        B    3  POINT (3.00000 2.00000)
